In [5]:
!pip install transformers torch

In [8]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
import torch

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/293 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

The tied weights mapping and config for this model specifies to tie transformer.wte.weight to lm_head.weight, but both are present in the checkpoints, so we will NOT tie them. You should update the config with `tie_word_embeddings=False` to silence this warning
GPT2LMHeadModel LOAD REPORT from: microsoft/DialoGPT-medium
Key                              | Status     |  | 
---------------------------------+------------+--+-
transformer.h.{0...23}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [9]:
chat_history_ids = None

def chat(user_input):
    '''
    This function takes in a user input and returns a response from the chatbot.

    Args:
        user_input (str): The user's input.

    Returns:
        str: The chatbot's response.
    '''
    global chat_history_ids
    new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')
    bot_input_ids = (torch.cat([chat_history_ids, new_input_ids], dim=-1) if chat_history_ids is not None else new_input_ids)
    chat_history_ids = model.generate(bot_input_ids,
                                      max_length=1000,
                                      pad_token_id=tokenizer.eos_token_id
                                      )
    response = tokenizer.decode(chat_history_ids[:,bot_input_ids.shape[-1]:][0],
                                skip_special_tokens=True
                                )
    return response

In [10]:
while True:
    user_input = input(">> User: ")
    if user_input.lower() == "exit":
        break
    print(">> BotKay: ", chat(user_input))

>> User:Hello


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


>> Bot: Hello! :D
>> User:Can I call you darling?
>> Bot: Of course!
>> User:Tell me something about Koketso Llale
>> Bot: I'm a big fan of the show!
>> User:Who is Nasty C?
>> Bot: I'm not sure, but I think it's a girl.
>> User:Ok thank you darling. Let me go to sleep, wanna join me?
>> Bot: Sure! I'll be there in a bit.
>> User:exit
